In [1]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [5]:
#Erstellen von costum Retrieval Systemen

def create_index(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords='terrier')
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

stopwords = [
  "a", "about", "above", "after", "again", "against", "all", "am", "an", "and",
  "any", "are", "aren't", "as", "at", "be", "because", "been", "before", "being",
  "below", "between", "both", "but", "by", "can't", "cannot", "could", "couldn't",
  "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down", "during",
  "each", "few", "for", "from", "further", "had", "hadn't", "has", "hasn't",
  "have", "haven't", "having", "he", "he'd", "he'll", "he's", "her", "here",
  "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i",
  "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "isn't", "it", "it's",
  "its", "itself", "let's", "me", "more", "most", "mustn't", "my", "myself",
  "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought",
  "our", "ours", "ourselves", "out", "over", "own", "same", "shan't", "she",
  "she'd", "she'll", "she's", "should", "shouldn't", "so", "some", "such", "than",
  "that", "that's", "the", "their", "theirs", "them", "themselves", "then",
  "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've",
  "this", "those", "through", "to", "too", "under", "until", "up", "very",
  "was", "wasn't", "we", "we'd", "we'll", "we're", "we've", "were", "weren't",
  "what", "what's", "when", "when's", "where", "where's", "which", "while",
  "who", "who's", "whom", "why", "why's", "with", "won't", "would", "wouldn't",
  "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
  "yourselves", 
    #Domain-specific stop words
  "algorithm", "analysis", "approach", "based", "data", "database", "document",
  "documents", "example", "examples", "framework", "information", "method",
  "methods", "model", "models", "paper", "papers", "query", "queries", "relevant",
  "retrieval", "search", "system", "systems", "technique", "techniques", "term",
  "terms", "text", "used", "using", "web"
]


costum_index = create_index(pt_dataset.get_corpus_iter(), stopwords)

XSqrA_M	 = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_costum	 = pt.BatchRetrieve(costum_index, wmodel="XSqrA_M")

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  49%|████▉     | 62351/126958 [00:08<00:07, 8613.51it/s]

11:13:32.914 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (P82-1020) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:15<00:00, 8303.97it/s] 


11:13:41.488 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 8 empty documents


In [6]:
#Evaluation

pt.Experiment(
    [XSqrA_M, XSqrA_M_costum],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names = ["XSqrA_M", "XSqrA_M_costum"]
)

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.448211,0.650289,0.592214,0.400000
1,XSqrA_M_costum,0.324959,0.539898,0.488470,0.280882
